In [1]:
from scipy.signal import butter, lfilter
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
import datetime
from zipfile import ZipFile

In [2]:
def get_utc_date_time(ts):
    return pd.to_datetime(ts, unit='s', infer_datetime_format = True, utc = True)
#.strftime('%H:%M:%S:%f')

def add_fs(sample_rate, date):
    return date + datetime.timedelta(milliseconds=1.0/(sample_rate) * 1000.0)

In [3]:
def time_range(df_length, start_timestamp, sample_rate):
    # Generate date time range based on sample rate and starting timestamp
    time_range = []
    t_0 = get_utc_date_time(float(start_timestamp))
    time_range.append(t_0)
    
    next_date = t_0
    
    for count in range(df_length-1):
        next_date = add_fs(sample_rate, next_date)
        time_range.append(next_date)
    return time_range

In [4]:
def format_time(t):
    s = t.strftime('%Y-%m-%d %H:%M:%S.%f')
    return s[:-3]

In [5]:
base_path = r"C:\Users\shaye\Documents\EAT Lab\Sensor_Data\Empatica data"
participant_id = "PR025"
base_path

'C:\\Users\\shaye\\Documents\\EAT Lab\\Sensor_Data\\Empatica data'

In [6]:
list_of_zips = glob.glob(base_path + '\A*.zip')

In [10]:
temp_all = []

for zip_file in list_of_zips:
    zf = ZipFile(zip_file)
    temp_df_raw = pd.read_csv(zf.open('TEMP.csv'))
    
    starting_timestamp = temp_df_raw.columns[0]
    sample_rate = temp_df_raw[starting_timestamp][0]
    
    temp_df_raw = temp_df_raw.drop([0, 0])
    temp_df = temp_df_raw.copy()
    
    time_col = time_range(len(temp_df_raw), starting_timestamp, sample_rate)
    temp_df['Timestamp'] = time_col
    temp_df['Timestamp'] = temp_df.apply(lambda x: format_time(x['Timestamp']) + '+00:00', axis=1)
    
    temp_df = temp_df.rename(columns={starting_timestamp:"TEMP"})
    temp_df['ParticipantId'] = participant_id
    
    temp_all.append(temp_df)

In [11]:
temp_df_all = pd.concat(temp_all)

In [12]:
temp_df_all

,TEMP,Timestamp,ParticipantId
1,35.84,2020-12-15 13:59:08.000+00:00,PR025
2,35.84,2020-12-15 13:59:08.250+00:00,PR025
3,35.84,2020-12-15 13:59:08.500+00:00,PR025
4,35.84,2020-12-15 13:59:08.750+00:00,PR025
5,35.84,2020-12-15 13:59:09.000+00:00,PR025
...,...,...,...
158788,35.95,2021-01-14 01:34:40.750+00:00,PR025
158789,35.95,2021-01-14 01:34:41.000+00:00,PR025
158790,35.95,2021-01-14 01:34:41.250+00:00,PR025
158791,35.95,2021-01-14 01:34:41.500+00:00,PR025
